# FMLIKH Model: Small Amplitude Oscillatory Shear (SAOS)

## Learning Objectives

1. Generate **synthetic SAOS data** from NB07 calibrated parameters
2. Analyze **broadened relaxation spectra** with multi-mode fractional behavior
3. Compare FMLIKH vs single-mode FIKH frequency response
4. Understand multi-Maxwell-like behavior with fractional modifications

## Prerequisites

- NB07: FMLIKH Flow Curve (calibrated parameters)
- NB05: FIKH SAOS (single-mode concepts)

## 1. Setup

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os, sys, time, warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

# Robust path resolution for execution from any directory
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    load_fikh_parameters, generate_synthetic_saos, save_fikh_results,
    set_model_parameters, print_convergence_summary, get_fmlikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()
warnings.filterwarnings("ignore", category=FutureWarning)

## 2. Load Calibrated Parameters

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

try:
    params = load_fikh_parameters("fmlikh", "flow_curve")
    set_model_parameters(model, params)
    print("Loaded parameters from NB07")
except FileNotFoundError:
    print("Using default parameters")

## 3. Generate Synthetic Data

In [ ]:
OMEGA_RANGE = (0.01, 100.0)
omega_data, G_prime_data, G_double_prime_data = generate_synthetic_saos(
    model, omega_range=OMEGA_RANGE, n_points=40, noise_level=0.03, seed=42
)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(omega_data, G_prime_data, "ko", markersize=7, label="G'")
ax.loglog(omega_data, G_double_prime_data, "s", color="gray", markersize=6, label="G''")
ax.set_xlabel("Frequency [rad/s]")
ax.set_ylabel("Modulus [Pa]")
ax.set_title("Synthetic FMLIKH SAOS Data")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Mode Contribution Analysis

In [ ]:
# Each mode has characteristic frequency 1/tau_i
mode_info = model.get_mode_info()
omega_fine = np.logspace(np.log10(OMEGA_RANGE[0]), np.log10(OMEGA_RANGE[1]), 100)

print("Mode characteristic frequencies:")
for m in mode_info["modes"]:
    omega_c = 1.0 / m["tau"] if m["tau"] > 0 else float("inf")
    print(f"  Mode {m['mode']}: ω_c = {omega_c:.4g} rad/s (τ = {m['tau']:.4g} s)")

## 5. Bayesian Inference

In [ ]:
model_fit = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.5)
G_star_data = G_prime_data + 1j * G_double_prime_data

# For SAOS, create synthetic time-domain oscillation data
# Use low frequency for fitting
omega_fit = np.median(omega_data)
period = 2 * np.pi / omega_fit
t_fit = np.linspace(0, 5 * period, 200)
gamma_0 = 0.01
strain_fit = gamma_0 * np.sin(omega_fit * t_fit)

# Use G_prime as proxy for stress magnitude (simplified)
stress_fit = G_prime_data[len(G_prime_data)//2] * strain_fit

model_fit.fit(t_fit, stress_fit, test_mode="startup", strain=strain_fit, method='scipy')

param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model_fit.parameters.get_value(n) for n in param_names if n in model_fit.parameters}

NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 200, 500, 1

t0 = time.time()
result = model_fit.fit_bayesian(
    t_fit, stress_fit, test_mode="startup", strain=strain_fit,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "G_1", "alpha_structure"]
print_convergence_summary(result, key_params)

## 6. Save Results

In [ ]:
save_fikh_results(model_fit, result, "fmlikh", "saos", param_names)

## Key Takeaways

1. **Multi-mode SAOS** shows broadened frequency response
2. **Each mode contributes** at its characteristic frequency
3. **Fractional order** modifies frequency dependence within each mode
4. **Wide-frequency data** required to constrain multiple modes